In [3]:
import panel as pn

pn.extension("vega") 

import numpy as np
import pandas as pd
import geopandas as gpd
import carto2gpd

import holoviews as hv
import hvplot.pandas

import param as pm

import altair as alt
import folium
from folium.plugins import MarkerCluster
from matplotlib import pyplot as plt

#!pip install pyepsg

In [4]:
# load the data
chicago_data = gpd.read_file('data/chicago_data.geojson').set_crs("epsg:3857", allow_override=True)
chicago_data.to_crs(epsg=4326, inplace=True)

bus_ridership = gpd.read_file('data/bus_ridership.geojson') #4326
#bus_ridership.to_crs("epsg:3857", inplace = True)


In [ ]:
columns = list(chicago_data.columns[2:-1])
class PlanRidership(pm.Parameterized):
    #x = pm.selector(default='boardings', Object=columns)
    #@pm.depend("x")
    def choropleth(self):
        # Minimum
        min_val = chicago_data['boardings'].min()

        # Maximum
        max_val = chicago_data['boardings'].max()

        # Calculate a normalized column
        normalized = (chicago_data['boardings'] - min_val) / (max_val - min_val)

        # Add to the dataframe
        chicago_data['boardings_normalized'] = normalized

        import matplotlib.colors as mcolors
        from matplotlib import pyplot as plt
        # Use a red-purple colorbrewer color scheme
        cmap = plt.get_cmap('RdPu')

        def get_style(feature):
            # Get the data value from the feature
            value = feature['properties']['boardings_normalized']

            # Evaluate the color map
            # NOTE: value must between 0 and 1
            rgb_color = cmap(value) # this is an RGB tuple

            # Convert to hex string
            color = mcolors.rgb2hex(rgb_color)

            # Return the style dictionary
            return {'weight': 0.5, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

        def get_highlighted_style(feature):
            return {"weight": 3, "color": "black"}
        
        m = folium.Map(
            location=[41.89051776994878, -87.62846577303873],
            zoom_start=11
        )
        
        folium.GeoJson(
            chicago_data.to_crs(epsg=4326).to_json(), # IMPORTANT: make sure CRS is lat/lng (EPSG=4326)
            name='Chicago ridership',
            style_function=get_style,
            highlight_function=get_highlighted_style,
            tooltip=folium.GeoJsonTooltip(['GEOID', 'boardings'])
        ).add_to(m)

        
        TOR_COORDINATES = (pd.to_numeric(bus_ridership['latitude']).mean(), pd.to_numeric(bus_ridership['longitude']).mean())

        # subset to match subset of locations
        MAX_RECORDS = 2500

        mc = MarkerCluster()

        # iterate through dataset to create clusters
        for row in bus_ridership[0:MAX_RECORDS].itertuples():
            mc.add_child(folium.Marker(location=[row.latitude,  row.longitude],
                         popup=row.boardings))

        m.add_child(mc)

        return pn.Pane(m, width=600, height=600)
    
    """
    x = pm.Selector(default='total_pop', objects=columns)
    y = pm.Selector(default='boardings', objects=columns)
    color = pm.Color(default='#0f0f0f')
    
    @pm.depends('x', 'y', 'color') # This is a Python "decorator"!
    def make_auto_plot(self):
        return chicago_data.hvplot.scatter(self.x, self.y, c=self.color, padding=0.1)
    """

In [ ]:
chicago_data

In [ ]:
app = PlanRidership(name="")

instructions = pn.Pane(
"""
<div font-size=28px>
    <b>Note:</b> Data for a specific time period can be selected by clicking
    and dragging a specific range on the line chart above.
</div>
""",
    width=1000,
)

panel = pn.template.BootstrapTemplate(
    title="Bus Ridership in Chicago", 
    sidebar_width=0 # No sidebar needed for this app
)  

In [ ]:
""""
panel.sidebar.append(
    pn.Column(
        pn.pane.Markdown("#### Viewing Options"),
        pn.Param(app.param, expand_button=False, width=200),
    )
)

panel.main.append(
    pn.Column(
        #pn.Row(app.summary_text, pn.Param(app.param, width=300)),
        pn.Row(app.choropleth, app.make_auto_plot),
    )
)
"""""

In [ ]:
#panel.servable()

In [5]:
def autompg_plot(x='tot_pop', y='boardings', color='#058805'):
    return chicago_data.hvplot.scatter(x, y, c=color, padding=0.1)
columns = list(chicago_data.columns[2:-1])

class MPGExplorer(pm.Parameterized):
    x = pm.Selector(default='total_pop', objects=columns)
    y = pm.Selector(default='boardings', objects=columns)
    color = pm.Color(default='#0f0f0f')
    
    @pm.depends('x', 'y', 'color') # This is a Python "decorator"!
    def make_autompg_plot(self):
        return autompg_plot(self.x, self.y, self.color)

explorer = MPGExplorer()


class_dashboard = pn.Row(explorer.param, explorer.make_autompg_plot)
class_dashboard

Row
    [0] Column(margin=5, name='MPGExplorer', width=300)
        [0] StaticText(value='<b>MPGExplorer</b>')
        [1] Select(name='X', options=OrderedDict([('total_pop',...]), value='total_pop')
        [2] Select(name='Y', options=OrderedDict([('total_pop',...]), value='boardings')
        [3] ColorPicker(name='Color', value='#0f0f0f')
    [1] ParamMethod(method)